# A1 — Classificação de tipo de terreno com LiDAR compacto (Fácil)

Classifique o tipo de terreno (liso, cascalho, grama) a partir de pequenas características de um patch de LiDAR usando um **kernel quântico** ou **VQC**.

### Contexto
**LiDAR (Light Detection and Ranging)** emite pulsos de laser e mede o tempo de retorno para construir uma nuvem de pontos 3D ou amostras localizadas de altura/intensidade. Pequenos patches voltados ao solo em um robô móvel ajudam a antecipar tração e vibração do terreno à frente.

Em robótica de campo, saber se a superfície adiante é asfalto liso, cascalho solto ou grama influencia:
- Planejamento de movimento (slip esperado / custo energético)
- Colocação de pés ou rodas e perfis de velocidade
- Controle antecipatório (alocação de torque, ajuste de marcha)

O conjunto de dados comprime cada patch em vetores de características engenheiradas (por exemplo, estatísticas de altura local, indicadores de rugosidade). Você recebe apenas esses vetores (sem nuvem bruta) para manter a dimensionalidade baixa (d=12) e tornar codificações quânticas viáveis (≤ 10 qubits).

### Por que métodos quânticos?
Fronteiras de decisão moderadamente não lineares em baixa dimensão são um regime plausível onde mapeamentos de características (feature maps) quânticos ou circuitos variacionais enriquecem o kernel sob orçamento de qubits restrito. Objetivo: projetar tal embedding ou classificador que generalize melhor que o SVM RBF clássico em seeds ocultos.

Meta: superar a acurácia do baseline no conjunto oculto com ≤ 10 qubits e ≤ 150 passos de otimização.

### Mini‑resumo de LiDAR
**LiDAR** emite pulsos curtos e mede tempo de voo para estimar alcance $r$ e, frequentemente, registra intensidade $I$. Padrões de varredura (rotativos ou solid‑state) geram amostras esparsas da microestrutura da superfície. Para uso voltado ao solo, um pequeno patch à frente reflete micro‑geometria e material:
- Asfalto liso: baixa variância de micro‑altura, baixa rugosidade, intensidade estável.
- Cascalho solto: variância e rugosidade mais altas; jitter de intensidade por múltiplos retornos.
- Grama: variância moderada com anisotropia (estruturas de lâminas) e intensidade tipo speckle.

Neste desafio, pré‑computamos características compactas (d=12) por patch para manter qubits pequenos, preservando pistas discriminativas (médias/variâncias, proxies de rugosidade/curvatura, energia de gradiente, espalho de intensidade) — suficiente para kernels quânticos ou VQCs pequenos.

### Geração de dados e características
Geramos vetores por classe com médias e variâncias específicas e embaralhamos para balancear. Veja [`common.data_utils.terrain_dataset`](common/data_utils.py) para a construção (ordem: liso → cascalho → grama). Isso enfatiza a qualidade do embedding em vez do processamento da nuvem bruta.

## Especificação de entrada e saída
**Entrada:** `X_train (N×d)`, `y_train (N,)`, `X_test (M×d)` com d=12 nos dados fornecidos  
**Rótulos:** inteiros em {0,1,2} mapeando para classes [0=liso, 1=cascalho, 2=grama]  
**Saída:** `y_pred (M,)` classes previstas em {0,1,2}  
**Tipo de retorno:** array de inteiros (ex.: `np.int64`)

Dicas de desempenho: mantenha ≤ 10 qubits e ≤ 150 passos de otimização; evite loops pesados em Python por amostra.

## Setup

In [ ]:
import numpy as np
from common import data_utils as du
from common import baselines as bl
from common import quantum_utils as qu
np.random.seed(1337)

## Baseline (referência)

In [ ]:
# Dados de exemplo
Xtr, ytr = du.terrain_dataset(n=240, d=12, seed=0)
Xte, yte = du.terrain_dataset(n=120, d=12, seed=101)

# Baseline: SVM RBF
yp = bl.baseline_terrain(Xtr, ytr, Xte)
acc = (yp == yte).mean()
print(f"Acurácia do baseline: {acc:.3f}")

## Sua tarefa: implementar `solve(...)`

In [ ]:
# Implemente um classificador com kernel quântico ou VQC.
# Assinatura: solve(Xtr: np.ndarray, ytr: np.ndarray, Xte: np.ndarray) -> np.ndarray
def solve(Xtr, ytr, Xte):
    # Stub: retorna o baseline para passar nos testes públicos.
    # Substitua por método quântico (ex.: SVM com feature map quântico).
    return bl.baseline_terrain(Xtr, ytr, Xte)

# teste rápido
# _ = solve(Xtr, ytr, Xte)

## Testes públicos (rápidos)

In [ ]:
# Testes públicos
Xtr, ytr = du.terrain_dataset(n=240, d=12, seed=1)
Xte, yte = du.terrain_dataset(n=120, d=12, seed=2)
yp = solve(Xtr, ytr, Xte)
acc = (yp == yte).mean()
print("Acurácia pública:", acc)
assert yp.shape == yte.shape
assert acc >= 0.70, "Limiar público não atingido"
print("OK")

> Testes adicionais serão executados pelos organizadores com seeds/tamanhos diferentes.